In [1]:
import pytorch_lightning as pl
import torch
import Operator as op
import utils
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [2]:
lattice_sites = 3
hamiltonian = []
h = -1
'''
for l in range(lattice_sites):
  hamiltonian = Sx(l)* (h) + Sz(l) * Sz((l+1) % lattice_sites) + hamiltonian
print_op_list(hamiltonian)
'''
hamiltonian = op.Sx(2) + ([op.Sz(1)*op.Sy(2)] + hamiltonian)
op.print_op_list(hamiltonian)

adding to sequence
Hamiltonan = Sx_2 + Sz_1 * Sy_2


In [3]:
import torch
from torch.nn import functional as F
import pytorch_lightning as pl
from torch import nn

class Model(pl.LightningModule):

  def __init__(self):
    super().__init__()
    self.lattice_struct = nn.Sequential(
      nn.Conv1d(1, 8, 2),
      nn.ReLu(),
      nn.Conv1d(8, 16),
      nn.Flatten()
    )
    
    self.tNN = nn.Sequential(
      nn.Linear(2, 16),
      nn.ReLu(),
      nn.Linear(16, 32),
      nn.ReLu(),
      nn.Linear(32, 64)
    )

    self.output = nn.Sequential(
      nn.Linear( 64 + )
    )


  def forward(self, x):
    x = self.layer_1(x)
    x = F.relu(x)
    x = self.layer_2(x)
    x = F.relu(x)
    x = self.layer_3(x)
    return x

model = Model().to(device)
print(model)

Model(
  (layer_1): Linear(in_features=3, out_features=128, bias=True)
  (layer_2): Linear(in_features=128, out_features=256, bias=True)
  (layer_3): Linear(in_features=256, out_features=1, bias=True)
)


In [4]:
from timeit import default_timer as timer

map = utils.get_map(hamiltonian, lattice_sites)
map = map.to(device)
print("map: ", map)

mat_els = utils.get_total_mat_els(hamiltonian, lattice_sites)
mat_els = mat_els.to(device)
print("mat els: ", mat_els)

#1.dim: Batch
#2.dim: lattice sites
#s_config = get_all_spin_configs(lattice_sites)
s_config = torch.tensor([1,1,1]).reshape(1,3)
s_config = (s_config.type(torch.float32)).to(device)
print("spin config: ", s_config)

start = timer()
s_p = utils.get_sp(s_config, map)
psi_s = model(s_config)
psi_sp = model(s_p.reshape(-1, lattice_sites)).reshape(s_p.shape[0], s_p.shape[1])
print(psi_sp.shape, psi_s.shape, s_config.shape)
O_loc = utils.calc_Oloc(psi_sp, mat_els, s_config)
end = timer()

print(f"time to calculate O_loc: {end - start:.2e}") 

map:  tensor([[[ 1,  1, -1],
         [ 1,  1, -1]]], device='cuda:0', dtype=torch.int8)
mat els:  tensor([[[[ 1.+0.j,  1.+0.j,  1.+0.j],
          [ 1.+0.j,  1.+0.j,  0.+1.j]],

         [[ 1.+0.j,  1.+0.j,  1.+0.j],
          [ 1.+0.j, -1.+0.j, -0.-1.j]]]], device='cuda:0')
spin config:  tensor([[1., 1., 1.]], device='cuda:0')
torch.Size([1, 2]) torch.Size([1, 1]) torch.Size([1, 3])
time to calculate O_loc: 4.15e-02


In [19]:
input = torch.arange(1,16,1)
input = input.reshape(3,1,5)
print(input)
padded_input = F.pad(input, (1,1), mode='circular')
print(padded_input)
#model = nn.Conv1d(1, 1, 2)
#model(input)

tensor([[[ 1,  2,  3,  4,  5]],

        [[ 6,  7,  8,  9, 10]],

        [[11, 12, 13, 14, 15]]])
tensor([[[ 5,  1,  2,  3,  4,  5,  1]],

        [[10,  6,  7,  8,  9, 10,  6]],

        [[15, 11, 12, 13, 14, 15, 11]]])


RuntimeError: expected scalar type Long but found Float